In [1]:
import sqlite3, config, datetime, numpy
import alpaca_trade_api as tradeapi
from datetime import date

In [2]:
# import sys
# !{sys.executable} -m pip install alpaca-trade-api

In [3]:
api = tradeapi.REST(config.API_KEY, config.SECRET_KEY, base_url=config.API_URL) 

We Like The Stock

In [ ]:
symbol = 'TSLA'

Dates

In [31]:
current_date = date.today()
d = datetime.timedelta(days=120)
from_date = (current_date - d).isoformat()
print(f"from:{from_date} -> to:{current_date}")

from:2020-10-22 -> to:2021-02-19


### daily price bars

In [29]:
barset = api.polygon.historic_agg_v2(symbol, 1, 'day', _from=from_date, to=current_date)
barset

[Agg({   'close': 425.79,
     'high': 445.23,
     'low': 424.51,
     'n': 696429,
     'open': 441.92,
     'timestamp': 1603339200000,
     'volume': 39993191.0,
     'vwap': 432.0209}),
 Agg({   'close': 420.63,
     'high': 422.8859,
     'low': 407.3801,
     'n': 716931,
     'open': 421.84,
     'timestamp': 1603425600000,
     'volume': 33611380.0,
     'vwap': 414.8284}),
 Agg({   'close': 420.28,
     'high': 425.76,
     'low': 410,
     'n': 531452,
     'open': 411.63,
     'timestamp': 1603684800000,
     'volume': 28239161.0,
     'vwap': 417.3324}),
 Agg({   'close': 424.68,
     'high': 430.5,
     'low': 420.1,
     'n': 405454,
     'open': 423.76,
     'timestamp': 1603771200000,
     'volume': 22686506.0,
     'vwap': 424.8484}),
 Agg({   'close': 406.02,
     'high': 418.6,
     'low': 406,
     'n': 600776,
     'open': 416.48,
     'timestamp': 1603857600000,
     'volume': 25421409.0,
     'vwap': 410.9556}),
 Agg({   'close': 410.83,
     'high': 418.06,
   

## TD Ameritrade API

In [4]:
from tda.auth import easy_client
from tda.client import Client
from tda.streaming import StreamClient

import asyncio
import json
import config

client = easy_client(
        api_key=config.tda_api_key,
        redirect_uri=config.tda_redirect_uri,
        token_path=config.tda_token_path)

stream_client = StreamClient(client, account_id=config.tda_account_id)

async def read_stream():
    await stream_client.login()
    await stream_client.quality_of_service(StreamClient.QOSLevel.EXPRESS)

    # Always add handlers before subscribing because many streams start sending
    # data immediately after success, and messages with no handlers are dropped.
    stream_client.add_timesale_futures_handler(
            lambda msg: print(json.dumps(msg, indent=4)))
    await stream_client.timesale_futures_subs(['SPY'])

    while True:
        await stream_client.handle_message()

asyncio.run(read_stream())

RuntimeError: asyncio.run() cannot be called from a running event loop